### Download model

In [ ]:
%%punch_dependencies
model:io.models:isblue_pyspark:1.0.0
additional-pex:io.models:isblue_pyspark_pex:1.0.0

++ java -Xmx1g -Xms256m -Dlog4j.configurationFile=/punch/conf/log4j2/log4j2-stdout.xml -cp /punch/resourcectl.jar com.github.punchplatform.resourcectl.ResourceCtl -u http://artifacts-server.artifacts-server:4245/ download -r model:models:isblue:1.0.0


Resource model:models:isblue:1.0.0 downloaded to /usr/share/punch/artifacts/models/isblue/1.0.0/isblue_1.0.0.zip


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


<IPython.core.display.Javascript object>

### Get model in a specified path

In [1]:
%punch_get_model --model io.models:isblue_pyspark:1.0.0 --output model_path

UsageError: unrecognized arguments: --override


In [2]:
%%punch_source --type generator --name data -o
messages:
          - color: red
          - color: blue
          - color: green
          - color: yellow
          - color: black

22/11/02 08:55:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
INFO:SparkMonitorKernel:Client Connected ('127.0.0.1', 53166)


Data is available in data variable.


### Datascientist loads its model with the appropriate function

##### Load model as spark_udf

In [3]:
import mlflow
from pyspark.sql import SparkSession

__spark_session = SparkSession.builder.getOrCreate()
isblue = mlflow.pyfunc.spark_udf(spark=__spark_session, model_uri=model_path, result_type="double")

2022/10/21 14:12:21 WARNING mlflow.pyfunc: Calling `spark_udf()` with `env_manager="local"` does not recreate the same environment that was used during training, which may lead to errors or inaccurate predictions. We recommend specifying `env_manager="conda"`, which automatically recreates the environment that was used to train the model and performs inference in the recreated environment.


In [4]:
from pyspark.sql.functions import struct

udf_inputs = struct(*(data.columns))
data.withColumn("prediction", isblue(udf_inputs)).show()

+------+----------+
| color|prediction|
+------+----------+
|   red|       0.0|
|  blue|       1.0|
| green|       0.0|
|yellow|       0.0|
| black|       0.0|
+------+----------+



### Load model with pyspark pandas

In [5]:
import pyspark.pandas as pd
from pyspark.pandas.mlflow import load_model

isblue = load_model(model_uri=model_path, predict_type="double")

df = pd.DataFrame(data)
isblue.predict(df)

2022/10/21 14:12:45 WARNING mlflow.pyfunc: Calling `spark_udf()` with `env_manager="local"` does not recreate the same environment that was used during training, which may lead to errors or inaccurate predictions. We recommend specifying `env_manager="conda"`, which automatically recreates the environment that was used to train the model and performs inference in the recreated environment.


0    0.0
1    1.0
2    0.0
3    0.0
4    0.0
Name: udf(color), dtype: float64